In [1]:
# This is the import cell
import sys
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.lines import Line2D
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from mpl_toolkits.axes_grid1.inset_locator import (inset_axes, InsetPosition,
                                                  mark_inset)
import matplotlib.ticker as ticker
from collections import OrderedDict
from scipy import stats
from PIL import Image
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
import pylab as plot
first = True
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", 10)



# Here are my rc parameters for matplotlibf
fsize = 20
mpl.rc('font', serif='Helvetica Neue')
#mpl.rc('font', serif='Times New Roman')
mpl.rcParams.update({'font.size': fsize})
mpl.rcParams['figure.figsize'] = 3.2, 2.8
mpl.rcParams['figure.dpi'] = 500
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
mpl.rcParams['lines.linewidth'] = 1.5
mpl.rcParams['axes.linewidth'] = 1.5
# Set x tick params
mpl.rcParams['xtick.major.size'] = 4.5
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['xtick.minor.size'] = 3.
mpl.rcParams['xtick.minor.width'] = 1.25
# Set y tick params
mpl.rcParams['ytick.major.size'] = 4.5
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['ytick.minor.size'] = 3.
mpl.rcParams['ytick.minor.width'] = 1.25
mpl.rcParams['legend.fontsize']= 20.
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams["font.family"] = "Times New Roman"
# Load LaTeX and amsmath
# mpl.rc('text', usetex=True)
# mpl.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

In [2]:
# Get the current path
if first:
    parent = os.getcwd()
print(parent)
# Grab file names from data folder
dens = os.listdir('../../../../../../../../Volumes/External/txt_files/random_txt_files/Align_press_CoM')
try:
    dens.remove('.DS_Store')
except:
    print(".DS_Store not in directory")
    
# Grab the MCS data
pres = os.listdir('../../../../../../../../Volumes/External/txt_files/random_txt_files/Interpart_press')
try:
    pres.remove('.DS_Store')
except:
    print(".DS_Store not in directory")
    
# Grab the MCS data
pres2 = os.listdir('../../../../../../../../Volumes/External/txt_files/random_txt_files/BubComp')
try:
    pres2.remove('.DS_Store')
except:
    print(".DS_Store not in directory")
# Grab the MCS data
pres3 = os.listdir('../../../../../../../../Volumes/External/txt_files/random_txt_files/PhaseComp')
try:
    pres3.remove('.DS_Store')
except:
    print(".DS_Store not in directory")
# Grab the MCS data
pres4 = os.listdir('../../../../../../../../Volumes/External/txt_files/random_txt_files/radial')
try:
    pres4.remove('.DS_Store')
except:
    print(".DS_Store not in directory")    
    

/Users/nicklauersdorf/hoomd-blue/build/njlauersdorf/ABPs/post_proc/Jupyter
.DS_Store not in directory
.DS_Store not in directory
.DS_Store not in directory
.DS_Store not in directory
.DS_Store not in directory


In [3]:
# Some functions to get the relevant data from the filenames
def checkFile(fname, string):
    for i in range(len(fname)):
        if fname[i] == string[0]:
#             print"{} matches {}".format(fname[i], string[0])
            for j in range(1, len(string)):
                if (i + j) > (len(fname) - 1):
                    break
                elif fname[i + j] == string[j]:
#                     print"{} matches {}".format(fname[i+j], string[j])
                    if j == (len(string) - 1):
#                         print"Final match!"
                        return True
                else:
                    break
    return False
    
def txtValue(fname, string):
    out = ""
    index = 0
    for i in range(len(fname)):
        if fname[i] == string[0]:
            for j in range(1, len(string)):
                if (i + j) > (len(fname) - 1):
                    break
                elif fname[i + j] == string[j]:
                    if j == (len(string) - 1):
                        # Last index of search string
                        index = i + j
                else:
                    break
                        
    # First index of value
    index += 1
    mybool = True
    while mybool:
        if fname[index].isdigit():
            out = out + fname[index]
            index += 1
        elif fname[index] == ".":    
            if fname[index+1].isdigit():
                out = out + fname[index]
                index += 1
            else:
                mybool = False
        else:
            mybool = False
    return float(out)

# Sorting functions
def multiSort(arr1, arr2, arr3, arr4):
    """Sort an array the slow (but certain) way, returns original indices in sorted order"""
    # Doing this for PeR, PeS, xS in this case
    cpy1 = np.copy(arr1)
    cpy2 = np.copy(arr2)
    cpy3 = np.copy(arr3)
    cpy4 = np.copy(arr4)
    ind = np.arange(0, len(arr1))
    for i in range(len(cpy1)):
        for j in range(len(cpy1)):
            # Sort by first variable
            if cpy1[i] > cpy1[j] and i < j:
                # Swap copy array values
                cpy1[i], cpy1[j] = cpy1[j], cpy1[i]
                cpy2[i], cpy2[j] = cpy2[j], cpy2[i]
                cpy3[i], cpy3[j] = cpy3[j], cpy3[i]
                cpy4[i], cpy4[j] = cpy4[j], cpy4[i]
                # Swap the corresponding indices
                ind[i], ind[j] = ind[j], ind[i]
                
            # If first variable is equal, resort to second variable
            elif cpy1[i] == cpy1[j] and cpy2[i] > cpy2[j] and i < j:
                # Swap copy array values
                cpy1[i], cpy1[j] = cpy1[j], cpy1[i]
                cpy2[i], cpy2[j] = cpy2[j], cpy2[i]
                cpy3[i], cpy3[j] = cpy3[j], cpy3[i]
                cpy4[i], cpy4[j] = cpy4[j], cpy4[i]
                # Swap the corresponding indices
                ind[i], ind[j] = ind[j], ind[i]
                
            elif cpy1[i] == cpy1[j] and cpy2[i] == cpy2[j] and cpy3[i] > cpy3[j] and i < j:
                # Swap copy array values
                cpy1[i], cpy1[j] = cpy1[j], cpy1[i]
                cpy2[i], cpy2[j] = cpy2[j], cpy2[i]
                cpy3[i], cpy3[j] = cpy3[j], cpy3[i]
                cpy4[i], cpy4[j] = cpy4[j], cpy4[i]
                # Swap the corresponding indices
                ind[i], ind[j] = ind[j], ind[i]
            elif cpy1[i] == cpy1[j] and cpy2[i] == cpy2[j] and cpy3[i] == cpy3[j] and cpy4[i] > cpy4[j] and i < j:
                # Swap copy array values
                cpy1[i], cpy1[j] = cpy1[j], cpy1[i]
                cpy2[i], cpy2[j] = cpy2[j], cpy2[i]
                cpy3[i], cpy3[j] = cpy3[j], cpy3[i]
                cpy4[i], cpy4[j] = cpy4[j], cpy4[i]
                # Swap the corresponding indices
                ind[i], ind[j] = ind[j], ind[i]
    return ind

def indSort(arr1, arr2):
    """Take sorted index array, use to sort array"""
    # arr1 is array to sort
    # arr2 is index array
    cpy = np.copy(arr1)
    for i in range(len(arr1)):
        arr1[i] = cpy[arr2[i]]

In [4]:
# Grab parameters, sort them
chkStrings = ["pe", "pa", "pb", "xa", "eps", "phi", "cluster", "dtau"]
default = [0., 0., 0., 100., 1., 60., 0, 0.000001]
storeVals = [[] for i in chkStrings]
for i in dens:
    for j in range(0, len(chkStrings)):
        if chkStrings[j] != "cluster":
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(txtValue(i, chkStrings[j]))
            else:
                storeVals[j].append(default[j])  
        else:
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(1)
            else:
                storeVals[j].append(default[j]) 
                
# Issue with epsilon in file output 0 -> 0.0001
for i in range(0, len(storeVals[4])):
    if storeVals[4][i] == 0.0:
        storeVals[4][i] = 0.0001

# Sort the arrays
if len(storeVals[0]) > 1:
    # Sort them!
#     print("Sorting... ")
    # Sort by: pe, phi, epsilon, cluster
    indArr = multiSort(storeVals[chkStrings.index("pa")],
                       storeVals[chkStrings.index("phi")],
                       storeVals[chkStrings.index("eps")],
                       storeVals[chkStrings.index("cluster")])
    indSort(dens, indArr)
    for i in storeVals:
        indSort(i, indArr)
    
# Now that the data is sorted, read it into a dataframe
all_dens = []
os.chdir(parent)
os.chdir('../../../../../../../../Volumes/External/txt_files/random_txt_files/Align_press_CoM')
for i in dens:
#     print(i)
    df = pd.read_csv(i, sep='\s+', header=0)
    all_dens.append(df)
os.chdir(parent)

# This is how you access the data at different levels
display(all_dens[0])

,tauB,sizeBin,clust_size,press_align,press_alignA,press_alignB
0,0.0,5.016308,16892,2932.054533,1005.300848,1926.753685
1,0.1,5.016308,16699,2441.541657,820.029821,1621.511836
2,0.2,5.016308,17057,2014.229723,652.761850,1361.467872
3,0.3,5.016308,17354,1604.733623,528.488354,1076.245269
4,0.4,5.016308,17644,1277.704774,422.123454,855.581320
...,...,...,...,...,...,...
994,99.4,5.016308,17347,295.628902,62.027328,233.601574
995,99.5,5.016308,17377,317.418539,61.589742,255.828796
996,99.6,5.016308,17457,251.418580,75.857726,175.560854
997,99.7,5.016308,16897,311.443815,71.814552,239.629262


In [5]:
# Grab the parameters from each file, store in a dataframe
headers = ['pe', 'peA', 'peB', 'xA', 'eps', 'phi', 'tauPer_dt']
params = pd.DataFrame(columns=headers)
for i in range(0, len(all_dens)):
    pe = int(storeVals[chkStrings.index("pe")][i])
    pa = int(storeVals[chkStrings.index("pa")][i])
    pb = int(storeVals[chkStrings.index("pb")][i])
    xa = float(storeVals[chkStrings.index("xa")][i])
    ep = float(storeVals[chkStrings.index("eps")][i])
    phi = float(storeVals[chkStrings.index("phi")][i])
    dtau = float(storeVals[chkStrings.index("dtau")][i])
    df = pd.DataFrame([[pe, pa, pb, xa, ep, phi, dtau]], columns=headers)
    params = params.append(df, ignore_index = True)
display(params)

,pe,peA,peB,xA,eps,phi,tauPer_dt
0,0,50,100,50.0,1.0,60.0,0.000001
1,0,50,150,50.0,1.0,60.0,0.000001
2,0,50,200,50.0,1.0,60.0,0.000001
3,0,50,250,50.0,1.0,60.0,0.000001
4,0,50,350,50.0,1.0,60.0,0.000001
...,...,...,...,...,...,...,...
23,0,250,250,50.0,1.0,60.0,0.000001
24,0,250,350,50.0,1.0,60.0,0.000001
25,0,350,350,50.0,1.0,60.0,0.000001
26,0,350,450,50.0,1.0,60.0,0.000001


In [6]:
# Let's add columns to the time-resolved simulation data
for i in range(len(all_dens)):
    # Ger rid of NaN in favor of 0
    all_dens[i].fillna(0, inplace=True)

headers=list(all_dens[0])
display(all_dens[0])
print(all_dens[0][headers[1]][0])
print(all_dens[0][headers[2]][0])

,tauB,sizeBin,clust_size,press_align,press_alignA,press_alignB
0,0.0,5.016308,16892,2932.054533,1005.300848,1926.753685
1,0.1,5.016308,16699,2441.541657,820.029821,1621.511836
2,0.2,5.016308,17057,2014.229723,652.761850,1361.467872
3,0.3,5.016308,17354,1604.733623,528.488354,1076.245269
4,0.4,5.016308,17644,1277.704774,422.123454,855.581320
...,...,...,...,...,...,...
994,99.4,5.016308,17347,295.628902,62.027328,233.601574
995,99.5,5.016308,17377,317.418539,61.589742,255.828796
996,99.6,5.016308,17457,251.418580,75.857726,175.560854
997,99.7,5.016308,16897,311.443815,71.814552,239.629262


5.016308
16892


In [7]:
# Grab parameters, sort them
chkStrings = ["pe", "pa", "pb", "xa", "eps", "phi", "cluster", "dtau"]
default = [0., 0., 0., 100., 1., 60., 0, 0.000001]
storeVals = [[] for i in chkStrings]
for i in pres:
    for j in range(0, len(chkStrings)):
        if chkStrings[j] != "cluster":
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(txtValue(i, chkStrings[j]))
            else:
                storeVals[j].append(default[j])  
        else:
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(1)
            else:
                storeVals[j].append(default[j]) 

# Issue with epsilon in file output 0 -> 0.0001
for i in range(0, len(storeVals[4])):
    if storeVals[4][i] == 0.0:
        storeVals[4][i] = 0.0001

# Sort the arrays
if len(storeVals[0]) > 1:
    # Sort them!
#     print("Sorting... ")
    # Sort by: pe, phi, epsilon, cluster
    indArr = multiSort(storeVals[chkStrings.index("pa")],
                       storeVals[chkStrings.index("phi")],
                       storeVals[chkStrings.index("eps")],
                       storeVals[chkStrings.index("cluster")])
    indSort(pres, indArr)
    for i in storeVals:
        indSort(i, indArr)
    
# Now that the data is sorted, read it into a dataframe
all_pres = []
os.chdir(parent)
os.chdir('../../../../../../../../Volumes/External/txt_files/random_txt_files/Interpart_press')
for i in pres:
#     print(i)
    df = pd.read_csv(i, sep='\s+', header=0)
    all_pres.append(df)
os.chdir(parent)

# This is how you access the data at different levels
display(all_pres[-1])

,Timestep,gasArea,gasSigXX,gasSigXY,gasSigYX,gasSigYY,gasTrace,bulkArea,bulkSigXX,bulkSigXY,bulkSigYX,bulkSigYY,bulkTrace,Length,NDense
0,0.0,38018.311,2375.154,-28.372,-28.372,2414.880,2366.645,25095.743,7.128161e+07,14131.687,14131.687,7.124745e+07,7.127866e+07,521.8,45750
1,0.1,44513.889,77255.194,3252.828,3252.828,78421.514,81091.182,18831.334,4.968946e+08,2209483.077,2209483.077,4.910797e+08,4.961966e+08,459.8,46021
2,0.2,43622.239,59034.237,-1976.006,-1976.006,50884.923,52983.574,19571.835,4.190462e+08,1208557.691,1208557.691,4.118017e+08,4.166325e+08,509.4,46458
3,0.3,42930.004,56950.722,-906.442,-906.442,50310.785,52724.311,20237.396,3.434012e+08,716554.830,716554.830,3.383948e+08,3.416146e+08,522.9,46576
4,0.4,42383.837,73100.650,1045.576,1045.576,66369.749,70780.776,20688.302,2.843977e+08,1262609.309,1262609.309,2.775112e+08,2.822171e+08,535.3,46367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,99.5,41653.497,230353.235,-1645.057,-1645.057,239412.599,233237.860,21164.610,1.383443e+08,-1680896.080,-1680896.080,1.403782e+08,1.376803e+08,597.3,43282
996,99.6,41767.811,241563.387,10528.837,10528.837,247869.621,255245.340,21047.756,1.410715e+08,-2053609.948,-2053609.948,1.400297e+08,1.384970e+08,604.1,43256
997,99.7,41824.969,256338.310,2767.425,2767.425,232529.161,247201.161,20985.518,1.400721e+08,-1738228.259,-1738228.259,1.408948e+08,1.387452e+08,591.7,43140
998,99.8,41732.247,250524.872,-7214.908,-7214.908,226997.909,231546.482,21055.377,1.383363e+08,-1196547.279,-1196547.279,1.377619e+08,1.368526e+08,618.7,43189


In [8]:
# Grab the parameters from each file, store in a dataframe
headers = ['pe', 'peA', 'peB', 'xA', 'eps', 'phi', 'tauPer_dt']
params2 = pd.DataFrame(columns=headers)
for i in range(0, len(all_pres)):
    pe = int(storeVals[chkStrings.index("pe")][i])
    pa = int(storeVals[chkStrings.index("pa")][i])
    pb = int(storeVals[chkStrings.index("pb")][i])
    xa = float(storeVals[chkStrings.index("xa")][i])
    ep = float(storeVals[chkStrings.index("eps")][i])
    phi = float(storeVals[chkStrings.index("phi")][i])
    dtau = float(storeVals[chkStrings.index("dtau")][i])
    df = pd.DataFrame([[pe, pa, pb, xa, ep, phi, dtau]], columns=headers)
    params2 = params2.append(df, ignore_index = True)
display(params2)

,pe,peA,peB,xA,eps,phi,tauPer_dt
0,0,50,100,50.0,1.0,60.0,0.000001
1,0,50,150,50.0,1.0,60.0,0.000001
2,0,50,200,50.0,1.0,60.0,0.000001
3,0,50,250,50.0,1.0,60.0,0.000001
4,0,50,350,50.0,1.0,60.0,0.000001
...,...,...,...,...,...,...,...
23,0,250,250,50.0,1.0,60.0,0.000001
24,0,250,350,50.0,1.0,60.0,0.000001
25,0,350,350,50.0,1.0,60.0,0.000001
26,0,350,450,50.0,1.0,60.0,0.000001


In [9]:
# Let's add columns to the time-resolved simulation data
for i in range(len(all_pres)):
    # Ger rid of NaN in favor of 0
    all_pres[i].fillna(0, inplace=True)

headers=list(all_pres[0])
display(all_pres[0])
print(all_pres[0][headers[1]][0])
print(all_pres[0][headers[2]][0])

,Timestep,gasArea,gasSigXX,gasSigXY,gasSigYX,gasSigYY,gasTrace,bulkArea,bulkSigXX,bulkSigXY,bulkSigYX,bulkSigYY,bulkTrace,Length,NDense
0,0.0,52372.344,245804.899,-3150.041,-3150.041,246134.235,242819.526,11327.252,5.272569e+06,-203.584,-203.584,5.262926e+06,5.267544e+06,410.2,16892
1,0.1,53958.769,1617559.425,-7098.746,-7098.746,1622842.356,1613102.144,9724.315,1.588969e+07,-21004.236,-21004.236,1.584887e+07,1.584828e+07,421.5,16699
2,0.2,54092.136,1676876.394,-10571.435,-10571.435,1632376.670,1644055.097,9556.654,2.109270e+07,-102464.400,-102464.400,2.114832e+07,2.101804e+07,436.2,17057
3,0.3,53791.109,1657353.338,-7777.292,-7777.292,1667794.953,1654796.854,9791.633,2.007455e+07,-77016.190,-77016.190,2.025556e+07,2.008804e+07,454.2,17354
4,0.4,53379.578,1658159.116,-5336.601,-5336.601,1637828.525,1642657.219,10148.547,1.777421e+07,-39972.993,-39972.993,1.800278e+07,1.784852e+07,459.8,17644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,99.5,49736.771,1807507.834,-939.918,-939.918,1791652.018,1798640.008,10762.032,5.026695e+06,47964.370,47964.370,4.960330e+06,5.041477e+06,1280.3,17377
996,99.6,49800.278,1804091.507,-8280.622,-8280.622,1772719.384,1780124.824,10844.592,4.962375e+06,8638.940,8638.940,4.984672e+06,4.982162e+06,1238.6,17457
997,99.7,50591.586,1861105.485,-10232.867,-10232.867,1843050.378,1841845.065,10637.557,5.007617e+06,-25962.821,-25962.821,4.976827e+06,4.966259e+06,1087.6,16897
998,99.8,50813.863,1881514.620,-1365.602,-1365.602,1871728.541,1875255.979,10647.718,5.051246e+06,18020.538,18020.538,5.009807e+06,5.048547e+06,1044.7,16870


52372.344000000005
245804.89899999998


In [10]:
# Grab parameters, sort them
chkStrings = ["pe", "pa", "pb", "xa", "eps", "phi", "cluster", "dtau"]
default = [0., 0., 0., 100., 1., 60., 0, 0.000001]
storeVals = [[] for i in chkStrings]
for i in pres2:
    for j in range(0, len(chkStrings)):
        if chkStrings[j] != "cluster":
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(txtValue(i, chkStrings[j]))
            else:
                storeVals[j].append(default[j])  
        else:
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(1)
            else:
                storeVals[j].append(default[j]) 

# Issue with epsilon in file output 0 -> 0.0001
for i in range(0, len(storeVals[4])):
    if storeVals[4][i] == 0.0:
        storeVals[4][i] = 0.0001

# Sort the arrays
if len(storeVals[0]) > 1:
    # Sort them!
#     print("Sorting... ")
    # Sort by: pe, phi, epsilon, cluster
    indArr = multiSort(storeVals[chkStrings.index("pa")],
                       storeVals[chkStrings.index("phi")],
                       storeVals[chkStrings.index("eps")],
                       storeVals[chkStrings.index("cluster")])
    indSort(pres2, indArr)
    for i in storeVals:
        indSort(i, indArr)
    
# Now that the data is sorted, read it into a dataframe
all_pres_new = []
os.chdir(parent)
os.chdir('../../../../../../../../Volumes/External/txt_files/random_txt_files/BubComp')
for i in pres2:
#     print(i)
    df = pd.read_csv(i, sep='\s+', header=0)
    all_pres_new.append(df)
os.chdir(parent)

# This is how you access the data at different levels
display(all_pres_new[-1])
print(storeVals)

,tauB,sizeBin,clust_size,int_id,bub_id,Na,Nb,radius,radius_err,sa_ext,sa_int,edge_width,edge_width_err,edge_begin,edge_end,NBin
0,0.0,5.016308,45750,1,1,15705,15515,93.914608,1.861855,611.364629,404.927523,39.430506,4.316719,54.484101,93.914608,853
1,0.1,5.016308,46021,1,1,14850,14697,82.884619,1.377431,541.019989,320.401332,33.372678,1.535488,49.511941,82.884619,660
2,0.2,5.016308,46458,1,1,14856,14663,84.236476,1.188504,547.059434,326.715860,33.437139,1.629779,50.799337,84.236476,673
3,0.3,5.016308,46576,1,1,14595,14554,84.676592,1.529403,551.143905,338.060060,32.391782,1.639879,52.284810,84.676592,663
4,0.4,5.016308,46367,1,1,14517,14504,85.866414,1.841334,562.662630,348.089928,32.670564,2.541268,53.195850,85.866414,682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004,99.4,5.016308,43296,1,1,9543,9403,88.673112,6.679194,600.382238,481.471927,21.521177,4.849941,67.151935,88.673112,555
1005,99.5,5.016308,43282,1,1,9584,9420,87.187941,6.807261,595.448710,474.988403,20.566850,5.468027,66.621091,87.187941,528
1006,99.6,5.016308,43256,1,1,9776,9651,87.878271,7.350654,612.547137,475.782221,21.736063,3.754774,66.142209,87.878271,555
1007,99.7,5.016308,43140,1,1,9168,9109,86.990295,6.840993,596.203878,476.399941,19.835932,8.229860,67.154363,86.990295,513


[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 150.0, 150.0, 150.0, 150.0, 150.0, 200.0, 200.0, 200.0, 200.0, 250.0, 250.0, 250.0, 350.0, 350.0, 450.0], [100.0, 150.0, 200.0, 250.0, 350.0, 450.0, 50.0, 100.0, 150.0, 200.0, 250.0, 350.0, 450.0, 150.0, 200.0, 250.0, 350.0, 450.0, 200.0, 250.0, 350.0, 450.0, 350.0, 450.0, 250.0, 450.0, 350.0, 450.0], [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 

In [11]:
# Grab the parameters from each file, store in a dataframe
headers = ['pe', 'peA', 'peB', 'xA', 'eps', 'phi', 'tauPer_dt']
params3 = pd.DataFrame(columns=headers)
for i in range(0, len(all_pres_new)):
    pe = int(storeVals[chkStrings.index("pe")][i])
    pa = int(storeVals[chkStrings.index("pa")][i])
    pb = int(storeVals[chkStrings.index("pb")][i])
    xa = float(storeVals[chkStrings.index("xa")][i])
    ep = float(storeVals[chkStrings.index("eps")][i])
    phi = float(storeVals[chkStrings.index("phi")][i])
    dtau = float(storeVals[chkStrings.index("dtau")][i])
    df = pd.DataFrame([[pe, pa, pb, xa, ep, phi, dtau]], columns=headers)
    params3 = params3.append(df, ignore_index = True)
display(params3)

,pe,peA,peB,xA,eps,phi,tauPer_dt
0,0,50,100,50.0,1.0,60.0,0.000001
1,0,50,150,50.0,1.0,60.0,0.000001
2,0,50,200,50.0,1.0,60.0,0.000001
3,0,50,250,50.0,1.0,60.0,0.000001
4,0,50,350,50.0,1.0,60.0,0.000001
...,...,...,...,...,...,...,...
23,0,250,450,50.0,1.0,60.0,0.000001
24,0,250,250,50.0,1.0,60.0,0.000001
25,0,350,450,50.0,1.0,60.0,0.000001
26,0,350,350,50.0,1.0,60.0,0.000001


In [12]:
# Let's add columns to the time-resolved simulation data
for i in range(len(all_pres_new)):
    # Ger rid of NaN in favor of 0
    all_pres_new[i].fillna(0, inplace=True)

headers=list(all_pres_new[0])
display(all_pres_new[0])
print(all_pres_new[0][headers[1]][0])
print(all_pres_new[0][headers[2]][0])

,tauB,sizeBin,clust_size,int_id,bub_id,Na,Nb,radius,radius_err,sa_ext,sa_int,edge_width,edge_width_err,edge_begin,edge_end,NBin
0,0.0,5.016308,16892,0,0,16656,16638,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2054
1,0.0,5.016308,16892,0,1,8344,8362,66.145435,0.923795,427.467884,0.000000,0.000000,0.000000,0.000000,0.000000,547
2,0.1,5.016308,16699,1,1,7851,7880,62.242037,1.965411,450.019642,147.813019,39.367783,2.664504,22.874254,62.242037,486
3,0.2,5.016308,17057,1,1,7798,7927,61.279127,1.920267,409.411172,148.014735,38.777544,1.954458,22.501583,61.279127,474
4,0.3,5.016308,17354,1,1,8252,8425,64.620688,2.418925,432.660735,145.337864,42.275733,2.716989,22.344955,64.620688,532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,21.4,5.016308,22259,1,1,11168,12529,93.999669,17.645407,822.800823,400.926600,58.758673,20.968717,35.240996,93.999669,987
986,21.4,5.016308,22259,1,7,405,361,5.094094,1.198970,34.543953,0.000000,0.000000,0.000000,0.000000,0.000000,48
987,21.4,5.016308,22259,1,9,187,189,5.616940,2.623527,59.505261,0.000000,0.000000,0.000000,0.000000,0.000000,25
988,21.4,5.016308,22259,1,10,118,119,7.477161,2.089270,52.199151,0.000000,0.000000,0.000000,0.000000,0.000000,15


5.016308
16892


In [13]:
# Grab parameters, sort them
chkStrings = ["pe", "pa", "pb", "xa", "eps", "phi", "cluster", "dtau"]
default = [0., 0., 0., 100., 1., 60., 0, 0.000001]
storeVals = [[] for i in chkStrings]
for i in pres3:
    for j in range(0, len(chkStrings)):
        if chkStrings[j] != "cluster":
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(txtValue(i, chkStrings[j]))
            else:
                storeVals[j].append(default[j])  
        else:
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(1)
            else:
                storeVals[j].append(default[j]) 

# Issue with epsilon in file output 0 -> 0.0001
for i in range(0, len(storeVals[4])):
    if storeVals[4][i] == 0.0:
        storeVals[4][i] = 0.0001

# Sort the arrays
if len(storeVals[0]) > 1:
    # Sort them!
#     print("Sorting... ")
    # Sort by: pe, phi, epsilon, cluster
    indArr = multiSort(storeVals[chkStrings.index("pa")],
                       storeVals[chkStrings.index("phi")],
                       storeVals[chkStrings.index("eps")],
                       storeVals[chkStrings.index("cluster")])
    indSort(pres3, indArr)
    for i in storeVals:
        indSort(i, indArr)
    
# Now that the data is sorted, read it into a dataframe
all_pres_new2 = []
os.chdir(parent)
os.chdir('../../../../../../../../Volumes/External/txt_files/random_txt_files/PhaseComp')
for i in pres3:
#     print(i)
    df = pd.read_csv(i, sep='\s+', header=0)
    all_pres_new2.append(df)
os.chdir(parent)

# This is how you access the data at different levels
display(all_pres_new2[-1])
print(storeVals)

,tauB,sizeBin,clust_size,Na_bulk,Nb_bulk,NBin_bulk,Na_gas,Nb_gas,NBin_gas,Na_int,Nb_int,NBin_int,Na_bub,Nb_bub,NBin_bub
0,0.0,5.016308,45750,7365,7510,326,1930,1975,1422,15705,15515,853,0,0,0
1,0.1,5.016308,46021,8296,8441,268,1854,1862,1673,14850,14697,660,0,0,0
2,0.2,5.016308,46458,8586,8734,284,1558,1603,1644,14856,14663,673,0,0,0
3,0.3,5.016308,46576,8831,8949,302,1574,1497,1636,14595,14554,663,0,0,0
4,0.4,5.016308,46367,8818,8945,311,1665,1551,1608,14517,14504,682,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,99.4,5.016308,43296,12496,12582,501,2961,3015,1545,9543,9403,555,0,0,0
995,99.5,5.016308,43282,12385,12456,494,3031,3124,1579,9584,9420,528,0,0,0
996,99.6,5.016308,43256,12236,12288,486,2988,3061,1560,9776,9651,555,0,0,0
997,99.7,5.016308,43140,12700,12710,504,3132,3181,1584,9168,9109,513,0,0,0


[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 150.0, 150.0, 150.0, 150.0, 150.0, 200.0, 200.0, 200.0, 200.0, 250.0, 250.0, 250.0, 350.0, 350.0, 450.0], [100.0, 150.0, 200.0, 250.0, 350.0, 450.0, 50.0, 100.0, 150.0, 200.0, 250.0, 350.0, 450.0, 150.0, 200.0, 250.0, 350.0, 450.0, 350.0, 450.0, 200.0, 250.0, 250.0, 350.0, 450.0, 350.0, 450.0, 450.0], [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 

In [14]:
# Grab the parameters from each file, store in a dataframe
headers = ['pe', 'peA', 'peB', 'xA', 'eps', 'phi', 'tauPer_dt']
params4 = pd.DataFrame(columns=headers)
for i in range(0, len(all_pres_new)):
    pe = int(storeVals[chkStrings.index("pe")][i])
    pa = int(storeVals[chkStrings.index("pa")][i])
    pb = int(storeVals[chkStrings.index("pb")][i])
    xa = float(storeVals[chkStrings.index("xa")][i])
    ep = float(storeVals[chkStrings.index("eps")][i])
    phi = float(storeVals[chkStrings.index("phi")][i])
    dtau = float(storeVals[chkStrings.index("dtau")][i])
    df = pd.DataFrame([[pe, pa, pb, xa, ep, phi, dtau]], columns=headers)
    params4 = params4.append(df, ignore_index = True)
display(params4)

,pe,peA,peB,xA,eps,phi,tauPer_dt
0,0,50,100,50.0,1.0,60.0,0.000001
1,0,50,150,50.0,1.0,60.0,0.000001
2,0,50,200,50.0,1.0,60.0,0.000001
3,0,50,250,50.0,1.0,60.0,0.000001
4,0,50,350,50.0,1.0,60.0,0.000001
...,...,...,...,...,...,...,...
23,0,250,350,50.0,1.0,60.0,0.000001
24,0,250,450,50.0,1.0,60.0,0.000001
25,0,350,350,50.0,1.0,60.0,0.000001
26,0,350,450,50.0,1.0,60.0,0.000001


In [15]:
# Let's add columns to the time-resolved simulation data
for i in range(len(all_pres_new2)):
    # Ger rid of NaN in favor of 0
    all_pres_new2[i].fillna(0, inplace=True)

headers=list(all_pres_new2[0])
display(all_pres_new2[0])
print(all_pres_new2[0][headers[1]][0])
print(all_pres_new2[0][headers[2]][0])

,tauB,sizeBin,clust_size,Na_bulk,Nb_bulk,NBin_bulk,Na_gas,Nb_gas,NBin_gas,Na_int,Nb_int,NBin_int,Na_bub,Nb_bub,NBin_bub
0,0.0,5.016308,16892,0,0,0,16656,16638,2054,16656,16638,2054,-8312,-8276,-1507
1,0.1,5.016308,16699,1066,999,49,16083,16121,2066,7851,7880,486,0,0,0
2,0.2,5.016308,17057,1107,1043,47,16095,16030,2080,7798,7927,474,0,0,0
3,0.3,5.016308,17354,1080,1016,46,15668,15559,2023,8252,8425,532,0,0,0
4,0.4,5.016308,17644,1135,1079,50,15297,15138,1976,8568,8783,575,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,21.1,5.016308,22040,2520,2857,151,10870,9453,1385,9595,10960,819,2015,1730,246
212,21.2,5.016308,22158,3051,3389,181,10945,9510,1382,9161,10454,804,1843,1647,234
213,21.3,5.016308,22257,3322,3760,200,10201,8793,1299,10810,11856,1013,667,591,89
214,21.4,5.016308,22259,2295,2556,136,10353,8858,1331,11168,12529,987,1184,1057,147


5.016308
16892


In [16]:
# Grab parameters, sort them
chkStrings = ["pe", "pa", "pb", "xa", "eps", "phi", "cluster", "dtau"]
default = [0., 0., 0., 100., 1., 60., 0, 0.000001]
storeVals = [[] for i in chkStrings]
for i in pres4:
    for j in range(0, len(chkStrings)):
        if chkStrings[j] != "cluster":
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(txtValue(i, chkStrings[j]))
            else:
                storeVals[j].append(default[j])  
        else:
            if checkFile(i, chkStrings[j]):
                storeVals[j].append(1)
            else:
                storeVals[j].append(default[j]) 

# Issue with epsilon in file output 0 -> 0.0001
for i in range(0, len(storeVals[4])):
    if storeVals[4][i] == 0.0:
        storeVals[4][i] = 0.0001

# Sort the arrays
if len(storeVals[0]) > 1:
    # Sort them!
#     print("Sorting... ")
    # Sort by: pe, phi, epsilon, cluster
    indArr = multiSort(storeVals[chkStrings.index("pa")],
                       storeVals[chkStrings.index("phi")],
                       storeVals[chkStrings.index("eps")],
                       storeVals[chkStrings.index("cluster")])
    indSort(pres4, indArr)
    for i in storeVals:
        indSort(i, indArr)
    
# Now that the data is sorted, read it into a dataframe
all_pres_new3 = []
os.chdir(parent)
os.chdir('../../../../../../../../Volumes/External/txt_files/random_txt_files/radial')
for i in pres4:
#     print(i)
    df = pd.read_csv(i, sep='\s+', header=0)
    all_pres_new3.append(df)
os.chdir(parent)

# This is how you access the data at different levels
display(all_pres_new3[-1])
print(storeVals)

,tauB,sizeBin,clust_size,interface_id,bub_id,r_min,r_max,align,alignA,alignB,num_dens,num_densA,num_densB,press,pressA,pressB
0,0.0,5.016308,2,1,0,0.000000,3.045615,-0.051717,-0.132185,0.060240,1.887395,1.098121,0.789274,-43.924345,-65.319849,21.395504
1,0.0,5.016308,2,1,0,3.045615,6.091230,-0.062963,-0.118521,-0.012825,1.784446,0.846468,0.937978,-50.559291,-45.146043,-5.413248
2,0.0,5.016308,2,1,0,6.091230,9.136846,-0.009930,-0.040910,0.015886,1.811899,0.823590,0.988309,-8.096709,-15.161960,7.065252
3,0.0,5.016308,2,1,0,9.136846,12.182461,0.021252,-0.022307,0.058311,1.823665,0.838297,0.985367,17.440739,-8.415143,25.855883
4,0.0,5.016308,2,1,0,12.182461,15.228076,-0.049118,0.008605,-0.108893,1.746317,0.888410,0.857907,-38.598912,3.440134,-42.039047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41953,99.8,5.016308,9,1,0,112.687764,115.733379,0.050449,0.022468,0.077612,0.154194,0.075953,0.078241,3.500524,0.767938,2.732586
41954,99.8,5.016308,9,1,0,115.733379,118.778994,-0.054521,-0.068269,-0.043598,0.143950,0.063730,0.080220,-3.531727,-1.957872,-1.573855
41955,99.8,5.016308,9,1,0,118.778994,121.824610,-0.093032,-0.154563,-0.039440,0.151165,0.070370,0.080795,-6.328443,-4.894478,-1.433965
41956,99.8,5.016308,9,1,0,121.824610,124.870225,0.004002,0.011936,-0.003187,0.146162,0.069480,0.076682,0.263228,0.373203,-0.109975


[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 150.0, 150.0, 150.0, 150.0, 150.0, 200.0, 200.0, 200.0, 200.0, 250.0, 250.0, 250.0, 350.0, 350.0, 450.0], [100.0, 150.0, 200.0, 250.0, 350.0, 450.0, 50.0, 100.0, 150.0, 200.0, 250.0, 350.0, 450.0, 150.0, 200.0, 250.0, 350.0, 450.0, 200.0, 250.0, 350.0, 450.0, 350.0, 450.0, 250.0, 350.0, 450.0, 450.0], [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 

In [17]:
# Grab the parameters from each file, store in a dataframe
headers = ['pe', 'peA', 'peB', 'xA', 'eps', 'phi', 'tauPer_dt']
params5 = pd.DataFrame(columns=headers)
for i in range(0, len(all_pres_new)):
    pe = int(storeVals[chkStrings.index("pe")][i])
    pa = int(storeVals[chkStrings.index("pa")][i])
    pb = int(storeVals[chkStrings.index("pb")][i])
    xa = float(storeVals[chkStrings.index("xa")][i])
    ep = float(storeVals[chkStrings.index("eps")][i])
    phi = float(storeVals[chkStrings.index("phi")][i])
    dtau = float(storeVals[chkStrings.index("dtau")][i])
    df = pd.DataFrame([[pe, pa, pb, xa, ep, phi, dtau]], columns=headers)
    params5 = params5.append(df, ignore_index = True)
display(params5)

,pe,peA,peB,xA,eps,phi,tauPer_dt
0,0,50,100,50.0,1.0,60.0,0.000001
1,0,50,150,50.0,1.0,60.0,0.000001
2,0,50,200,50.0,1.0,60.0,0.000001
3,0,50,250,50.0,1.0,60.0,0.000001
4,0,50,350,50.0,1.0,60.0,0.000001
...,...,...,...,...,...,...,...
23,0,250,450,50.0,1.0,60.0,0.000001
24,0,250,250,50.0,1.0,60.0,0.000001
25,0,350,350,50.0,1.0,60.0,0.000001
26,0,350,450,50.0,1.0,60.0,0.000001


In [18]:
# Let's add columns to the time-resolved simulation data
for i in range(len(all_pres_new3)):
    # Ger rid of NaN in favor of 0
    all_pres_new3[i].fillna(0, inplace=True)

headers=list(all_pres_new3[0])
display(all_pres_new3[0])
print(all_pres_new3[0][headers[1]][0])
print(all_pres_new3[0][headers[2]][0])

,tauB,sizeBin,clust_size,interface_id,bub_id,r_min,r_max,align,alignA,alignB,num_dens,num_densA,num_densB,press,pressA,pressB
0,0.0,5.016308,5,0,0,0.000000,3.045615,-0.060410,-0.012555,-0.130598,1.269702,0.754958,0.514744,-7.196370,-0.473938,-6.722432
1,0.0,5.016308,5,0,0,3.045615,6.091230,0.051455,0.165337,-0.058868,1.441283,0.709203,0.732080,1.553244,5.862868,-4.309624
2,0.0,5.016308,5,0,0,6.091230,9.136846,0.016156,-0.015971,0.053695,1.413830,0.761821,0.652009,2.892582,-0.608358,3.500939
3,0.0,5.016308,5,0,0,9.136846,12.182461,0.044373,0.018476,0.075691,1.397162,0.764763,0.632400,5.493170,0.706501,4.786670
4,0.0,5.016308,5,0,0,12.182461,15.228076,-0.047705,-0.024314,-0.069884,1.433657,0.697764,0.735893,-5.990987,-0.848279,-5.142708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41953,99.8,5.016308,315,8,4,112.687764,115.733379,-0.026698,-0.063798,0.016501,0.650636,0.350026,0.300611,-0.620507,-1.116549,0.496042
41954,99.8,5.016308,315,8,4,115.733379,118.778994,0.023790,-0.007055,0.061264,0.633737,0.347619,0.286117,1.630256,-0.122617,1.752873
41955,99.8,5.016308,315,8,4,118.778994,121.824610,-0.018157,-0.027815,-0.006921,0.655919,0.352719,0.303199,-0.700401,-0.490549,-0.209852
41956,99.8,5.016308,315,8,4,121.824610,124.870225,-0.005960,-0.013239,0.002670,0.674039,0.365617,0.308423,-0.159678,-0.242018,0.082340


5.016308
5


In [19]:
# All data is loaded, now compute analytical aspects
r_cut = (2.**(1./6.))

# Get lattice spacing for particle size
def ljForce(r, eps, sigma=1.):
    div = (sigma/r)
    dU = (24. * eps / sigma) * ((2*(div**13)) - (div)**7)
    return dU

# # Lennard-Jones pressure
# def ljPress(r, eps, sigma=1.):
#     phiCP = np.pi / (2. * np.sqrt(3.))
#     div = (sigma/r)
#     dU = (24. * eps / r) * ((2.*(div**12.)) - (div)**6.)
#     # This is just pressure divided by the area of a particle
# #     return (12. * dU / (np.pi * r))
#     return (12. * dU / (np.pi * r * phiCP))

def ljPress(r, pe, eps, sigma=1.):
    phiCP = np.pi / (2. * np.sqrt(3.))
    # This is off by a factor of 1.2...
    ljF = avgCollisionForce(pe)
    return (2. *np.sqrt(3) * ljF / r)
    
def avgCollisionForce(pe, power=1.):
    '''Computed from the integral of possible angles'''
    peCritical = 40.
    if pe < peCritical:
        pe = 0
    else:
        pe -= peCritical
    magnitude = 6.
    # A vector sum of the six nearest neighbors
    magnitude = np.sqrt(28)
#     return (magnitude * (pe**power)) / (np.pi)
#     return (pe * (1. + (8./(np.pi**2.))))
    coeff = 1.92#2.03#3.5#2.03
    #coeff= 0.4053
    return (pe * coeff)

def fStar(pe, epsilon, sigma=1.):
    out = (avgCollisionForce(pe) * sigma) / (24.*epsilon)
    return out
    
def conForRClust(pe, eps):
    out = []
    r = 1.112
    skip = [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]
    for j in skip:
        while ljForce(r, eps) < avgCollisionForce(pe):
            r -= j
        r += j
    out = r
    return out

def nonDimFLJ(r, sigma=1.):
    div = (sigma/r)
    dU = ((2*(div**13)) - (div)**7)
    return dU

def latForFStar(fstar):
    r = 1.112
    skip = [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]
    for j in skip:
        while nonDimFLJ(r) < fstar:
            r -= j
        r += j
    out = r
    return out
    
def latToPhi(latIn):
    '''Read in lattice spacing, output phi'''
    phiCP = np.pi / (2. * np.sqrt(3.))
    return phiCP / (latIn**2)

# From area fraction, get lattice spacing
def phiToLat(phiIn):
    '''Read in phi, output the lattice spacing'''
    phiCP = np.pi / (2. * np.sqrt(3.))
    latCP = 1.
    return np.sqrt(phiCP / phiIn)
    
def compPhiG(pe, a, kap=4.5, sig=1.):
    num = 3. * (np.pi**2) * kap * sig
    den = 4. * pe * a
    return num / den
    
def clustFrac(phi, phiG, a, sig=1.):
    phiL = latToPhi(a)
    ApL = np.pi * (sig**2) / 4.
    Ap = np.pi * (sig**2) / 4.
    num = (phiL*phiG) - (phiL*phi)
    den = (phi*phiG) - (phi*phiL)
    ans = num / den
    return ans

def radCurve(area):
    # From area of circle get curvature
    return np.sqrt(area/np.pi)

def radCirc(circ):
    return circ / (2. * np.pi)

In [20]:
#Constants
kT = 1.0                        # temperature
threeEtaPiSigma = 1.0           # drag coefficient
sigma = 1.0                     # particle diameter
D_t = kT / threeEtaPiSigma      # translational diffusion constant
D_r = (3.0 * D_t) / (sigma**2)  # rotational diffusion constant
tauBrown = (sigma**2) / D_t     # brownian time scale (invariant)



def compPeNet(xf, pes, pef):
    "Given each species activity (pes and pef) and particle fraction (xf), compute net activity (peNet)"
    peNet = (pes * (1.-xf)) + (pef * xf)
    return peNet
def avgCollisionForce(peNet):
    '''Computed from the integral of possible angles'''
    # A vector sum of the six nearest neighbors
    magnitude = np.sqrt(28)
    return (magnitude * peNet) / (np.pi)  
def ljForce(r, eps, sigma=1.):
    '''Compute the Lennard-Jones force'''
    div = (sigma/r)
    dU = (24. * eps / r) * ((2*(div**12)) - (div)**6)
    return dU

# Lennard-Jones pressure
def ljPress(r, pe, eps, sigma=1.):
    '''
    Purpose: Take epsilon (magnitude of lennard-jones force), sigma (particle diameter),
    activity (pe), and separation distance (r) of 2 particles to compute pressure from
    avg compressive active forces from neighbors
    
    Inputs: 
        r: Separation distance in simulation units
        epsilon: magnitude of lennard-jones potential
        pe: activity (peclet number)
        sigma: particle diameter (default=1.0)
    
    Output: Analytical virial pressure (see monodisperse paper for derivation)
    '''
    #Area fraction at HCP
    phiCP = np.pi / (2. * np.sqrt(3.))
    
    # LJ force
    ljF = avgCollisionForce(pe)
    
    return (2. *np.sqrt(3) * ljF / r)

def getLat(peNet, eps):
    '''Get the lattice spacing for any pe'''
    if peNet == 0:
        return 2.**(1./6.)
    out = []
    r = 1.112
    skip = [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]
    for j in skip:
        while ljForce(r, eps) < avgCollisionForce(peNet):
            r -= j
        r += j
    return r  
def latToPhi(latIn):
    '''Read in lattice spacing, output phi'''
    phiCP = np.pi / (2. * np.sqrt(3.))
    return phiCP / (latIn**2)

#Calculate gas phase area fraction
def compPhiG(pe, a, kap=4.5, sig=1.):
    '''
    Purpose: Compute analytical area fraction of the gas phase at steady state
    given activity and lattice spacing
    
    Inputs: 
        pe: net activity (peclet number)
        a: lattice spacing 
        kap: fitting parameter (default=4.5, shown by Redner)
        sig: particle diameter (default=1.0)
    
    Output: Area fraction of the gas phase at steady state
    '''
    num = 3. * (np.pi**2) * kap * sig
    den = 4. * pe * a
    if den>0:
        return num / den
    else:
        return 0
# Calculate dense phase area fraction from lattice spacing
def latToPhi(latIn):
    '''
    Purpose: Compute analytical area fraction of the dense phase given the lattice
    spacing.
    
    Inputs: 
        latIn: lattice spacing
    
    Output: dense phase area fraction
    '''
    phiCP = np.pi / (2. * np.sqrt(3.))
    return phiCP / (latIn**2)
def areaType(Nx, latx):
    Ax = Nx * np.pi * 0.25 * (latx**2)
    return Ax

#Slow activities interested in
pe_a = [0, 100, 150, 200, 250, 350, 450]
#Fast activities interested in
pe_b = [0, 100, 150, 200, 250, 350, 450]

#Particle fraction of slow activities
xA = 50./100.0

#Particle fraction of fast activities
xF = 50./100.0

#Particle fraction of slow activities
xS=1.0-xF

partNum = 50000          # total number of particles
partNumS=xS*partNum      # total number of slow particles
partNumF=xF*partNum      # total number of fast particles

intPhi = 60               # integer system area fraction
phi = float(intPhi)/100.0 # system area fraction
eps = 1.0                 # particle softness

#initialize empty arrays for appending to
press_fast_int = np.array([])
press_slow_int = np.array([])
press_fast_dense = np.array([])
press_slow_dense = np.array([])
pa_pair = np.array([])
pb_pair = np.array([])
pnet_pair = np.array([])
int_width_slow = np.array([])
int_width_fast = np.array([])
int_width_theory = np.array([])
int_width_predict_fastd = np.array([])
int_width_predict_slowd = np.array([])
int_width_fast_int_arr = np.array([])
int_width_slow_int_arr = np.array([])
#Loop over slow activities
for i in range(0, len(pe_a)): 
    
    #Loop over fast activities
    for j in range(0, len(pe_b)):
        
        #Be sure slow activity is less than or equal to fast activity
        if pe_a[i]<=pe_b[j]:
            
            #Compute net activity
            peNet = compPeNet(xF, pe_a[i], pe_b[j])
            
            # Compute lattice spacing based on each activity
            latS = getLat(pe_a[i], eps)
            latF = getLat(pe_b[j], eps)
            latNet = getLat(peNet, eps)
            
            Ns = partNum * (1. - xF)          # number of slow particles
            Nf = partNum - Ns                 # number of fast particles
            
            phiG = compPhiG(peNet, latNet)              # area fraction of gas phase
            NGas = (phiG / phi) * partNum    # number of gas particles
            
            #Compute area fraction of dense phase based on each lattice spacing
            phi_theory = latToPhi(latNet)
            phiS_theory = latToPhi(latS)
            phiF_theory = latToPhi(latF)
            
            #Compute number density of slow and fast particles
            nS_theory = phi_theory /(np.pi/4)
            nF_theory = phi_theory /(np.pi/4)
            
            #Compute number of particles in liquid phase
            Nl = int(round(partNum * ((phi_theory * (phiG - phi)) / (phi * (phiG - phi_theory)))))
            
            #Compute number of particles in gas phase
            NGas = partNum - Nl
            
            # Critical packing fraction in HCP lattice
            phiCP = np.pi / (2. * np.sqrt(3))
            
            # The area is the sum of the particle areas (normalized by close packing density of spheres)
            Al = (Nl * np.pi * (latNet)**2) / (4*phiCP)
            As = (Ns * np.pi * (latNet)**2) / (4*phiCP)
            if As > Al:
                As = Al
            Af = (Nf * np.pi * (latNet)**2) / (4*phiCP)
            if Af > Al:
                Af = Al
            
            # The area for instantiated liquid cluster
            Al_real=Al
            
            # The cluster radius is the square root of liquid area divided by pi
            Rl = np.sqrt(Al_real / np.pi)
            Rs = np.sqrt(As / np.pi)
            Rf = np.sqrt(Af / np.pi)
            
            #The interface width is the difference between the 100% bulk species radius and the total cluster radius
            int_width_fast_int = Rl-Rs
            int_width_slow_int = Rl-Rf
            int_width_fast_int_arr = np.append(int_width_fast_int_arr, int_width_fast_int)
            int_width_slow_int_arr = np.append(int_width_slow_int_arr, int_width_slow_int)
            #Calculate interface aligned active pressure from theory
            press_fast_int = np.append(press_fast_int, int_width_fast_int * nF_theory * 1.0 * pe_b[j])
            press_slow_int = np.append(press_slow_int, int_width_slow_int * nS_theory * 1.0 * pe_a[i])
            
            #Calculate bulk interparticle pressure from theory
            press_slow_dense_val = 4.0 * np.sqrt(3) * peNet / latNet
            press_fast_dense_val = 4.0 * np.sqrt(3) * peNet / latNet
            
            #Append pressures to array
            press_slow_dense = np.append(press_slow_dense, press_slow_dense_val)
            press_fast_dense = np.append(press_fast_dense, press_fast_dense_val)
            int_width_predict_slowd = np.append(int_width_predict_slowd, press_slow_dense_val/(pe_b[j]*nF_theory))
            int_width_predict_fastd = np.append(int_width_predict_fastd, press_fast_dense_val/(pe_a[i]*nS_theory))
            
            #Append activities to array
            pa_pair = np.append(pa_pair, pe_a[i])
            pb_pair = np.append(pb_pair, pe_b[j])
            pnet_pair = np.append(pnet_pair, pe_b[j]*0.5 + pe_a[i]*0.5)
            
            #Append interface widths to array
            int_width_fast = np.append(int_width_fast, int_width_fast_int)
            int_width_slow = np.append(int_width_slow, int_width_slow_int)
            
            #Calculate dense phase pressure again and constants for predicting what interface width is for balancing pressure
            curPLJ = ljPress(latNet, peNet, eps)

            alpha_max = 0.5
            I_arr = 3.0
            int_width = (np.sqrt(3)/(2*alpha_max)) * (curPLJ/peNet) * (latNet **2) * I_arr
            int_width_theory = np.append(int_width_theory, int_width)

<ipython-input-20-0d8571e0afad>:214: RuntimeWarning: invalid value encountered in double_scalars
  int_width_predict_slowd = np.append(int_width_predict_slowd, press_slow_dense_val/(pe_b[j]*nF_theory))
<ipython-input-20-0d8571e0afad>:215: RuntimeWarning: invalid value encountered in double_scalars
  int_width_predict_fastd = np.append(int_width_predict_fastd, press_fast_dense_val/(pe_a[i]*nS_theory))
<ipython-input-20-0d8571e0afad>:231: RuntimeWarning: invalid value encountered in double_scalars
  int_width = (np.sqrt(3)/(2*alpha_max)) * (curPLJ/peNet) * (latNet **2) * I_arr
<ipython-input-20-0d8571e0afad>:194: RuntimeWarning: invalid value encountered in sqrt
  Rl = np.sqrt(Al_real / np.pi)
<ipython-input-20-0d8571e0afad>:195: RuntimeWarning: invalid value encountered in sqrt
  Rs = np.sqrt(As / np.pi)
<ipython-input-20-0d8571e0afad>:196: RuntimeWarning: invalid value encountered in sqrt
  Rf = np.sqrt(Af / np.pi)
<ipython-input-20-0d8571e0afad>:215: RuntimeWarning: divide by zero enc

In [21]:
# Let's use analytical theory and kinetic theory to get cluster radius
epsRange = [1., 0.1, 0.01, 0.001, 0.0001]
# epsRange = [0.0001, 0.001, 0.01, 0.1, 1.]
peRange = np.arange(0., 700., 1.)
phiRange = [0.45, 0.55, 0.65]
N = 100000.
norm = 10.**0.
# norm = 1.

phiCP = np.pi / (2. * np.sqrt(3))
lat = []
pColl = []
pLJ = []
cfs = []
Rls = []
peCrit = []
phiGs = []
phiCPs = []
a_box = []
l_box = []
for b in range(0, len(phiRange)):
    lat.append([])
    phiCPs.append([])
    pColl.append([])
    pLJ.append([])
    cfs.append([])
    Rls.append([])
    phiGs.append([])
    peCrit.append([])
    a_box.append(N * np.pi * 0.25 / phiRange[b])
    l_box.append(np.sqrt(a_box[-1]))
    for i in range(0, len(epsRange)):
        lat[b].append([])
        phiCPs[b].append([])
        pColl[b].append([])
        pLJ[b].append([])
        cfs[b].append([])
        Rls[b].append([])
        phiGs[b].append([])
        for j in range(0, len(peRange)):
            # Compute lattice spacing
        
            lat[b][i].append(conForRClust(peRange[j], epsRange[i]))
            phiCPs[b][i].append(latToPhi(lat[b][i][-1]))
            # Compute pressure
            #curPLJ = ljPress(lat[b][i][-1], epsRange[i]) / (np.pi * (lat[b][i][-1]**2) * 0.25 * phiCP)
            #curPLJ = ljPress(lat[b][i][-1], epsRange[i]) / (np.pi * (lat[b][i][-1]**2) * 0.25 * phiCP * (lat[b][i][-1]**(0.5)) * 1.25)
            curPLJ = ljPress(lat[b][i][-1], peRange[j], epsRange[i])
            
            # Append to list
            pLJ[b][i].append(curPLJ/(norm))

            # Compute cluster fraction
            phiG = compPhiG(peRange[j], lat[b][i][-1])
            phiGs[b][i].append(phiG)
            if peRange[j] > 35.:
                cf = clustFrac(phiRange[b], phiG, lat[b][i][-1])
                if cf < 0. or cf > 1.:
                    cf = 0.
            else:
                cf = 0
            cfs[b][i].append(cf)

            # Get the critical activity
            if j > 0:
                if cfs[b][i][-2] == 0. and cfs[b][i][-1] > 0.:
                    peCrit[b].append(peRange[j])

            # Get the radius (for some N)
            Nl = cfs[b][i][-1] * N
            Al = Nl * ((np.pi * (lat[b][i][-1]**2))/(4*phiCP))
            Rl = np.sqrt(Al / (np.pi))
            Rls[b][i].append(Rl)

In [26]:
align_peA=np.array([])
align_peB=np.array([])
align_peNet=np.array([])
first_align_peNet=np.array([])
first_interpart_peNet=np.array([])
first_interpart_peA=np.array([])
first_interpart_peB=np.array([])

align_xA=np.array([])
align_eps=np.array([])
align_pnum=np.array([])
align_phi=np.array([])
align_press_arr = np.array([])
first_align_press = np.array([])
first_bulk_press = np.array([])
first_align_peA=np.array([])
first_align_peB=np.array([])

interpart_peA=np.array([])
interpart_peB=np.array([])
interpart_peNet=np.array([])
interpart_xA=np.array([])
interpart_eps=np.array([])
interpart_pnum=np.array([])
interpart_phi=np.array([])
interpart_press=np.array([])
interpart_press_expand=np.array([])
avg_shear=np.array([])
fastCol = '#e31a1c'
slowCol = '#081d58'
def ljForce(r, eps, sigma=1.):
    div = (sigma/r)
    dU = (24. * eps / sigma) * ((2*(div**13)) - (div)**7)
    return dU

def avgCollisionForce(pe, power=1.):
    '''Computed from the integral of possible angles'''
    peCritical = 40.
    if pe < peCritical:
        pe = 0
    else:
        pe -= peCritical
    magnitude = 6.
    # A vector sum of the six nearest neighbors
    magnitude = np.sqrt(28)
#     return (magnitude * (pe**power)) / (np.pi)
#     return (pe * (1. + (8./(np.pi**2.))))
    coeff = 1.874#3.0#1.92#2.03#3.5#2.03
    #coeff= 0.4053
    return (pe * coeff)

def conForRClust(pe, eps):
    out = []
    r = 1.112
    skip = [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]
    for j in skip:
        while ljForce(r, eps) < avgCollisionForce(pe):
            r -= j
        r += j
    out = r
    return out
def densProbability(r, activity_net, activity_slow):
    "Using similar triangles to find sides"
    gas_dense_dif_phi = 0.866 * np.log10(activity_net - 46.993) - 0.443
    rate_decay = -6.151 * np.log10(activity_net-49.921) - 4.392
    mid_point = 0.044 * np.log10(activity_slow-49.893) + 0.836
    gas_phi = -0.26 * np.log10(activity_slow-41.742)+0.783
    
    num_dens_r = ((gas_dense_dif_phi / (1+np.exp(-rate_decay * (r-mid_point)))) + gas_phi)
    
    return num_dens_r

def alignProbability(r, activity_net, activity_slow):
    "Using similar triangles to find sides"
    max_align = 0.2019 * np.log10(activity_net - 41.2803) - 0.1885
    mid_point = 0.0492 * np.log10(activity_slow - 47.0061) + 0.8220
    std_dev = 0.1057
    align_r = max_align * np.exp(-(r-mid_point)**2/(2*std_dev**2))
    
    return align_r

for i in range(0, len(all_dens)):
    if params['peA'][i]>=100:
        if all_dens[i].empty:
            continue
        for k in range(0, len(all_pres)):
            if params2['peA'][k]==params['peA'][i]:
                if params2['peB'][k]==params['peB'][i]:
                    if all_pres[k].empty:
                        continue
                    for q in range(0, len(all_pres_new2)):
                        if all_pres_new2[q].empty:
                            continue
                        if params3['peA'][q]==params['peA'][i]:
                            if params3['peB'][q]==params['peB'][i]:
                                for s in range(0, len(all_pres_new3)):
                                    if all_pres_new3[s].empty:
                                        continue
                                    if params5['peA'][s]==params['peA'][i]:
                                        if params5['peB'][s]==params['peB'][i]:
                                            for t in range(0, len(all_pres_new)):
                                                if all_pres_new[t].empty:
                                                    continue
                                                if params3['peA'][t]==params['peA'][i]:
                                                    if params3['peB'][t]==params['peB'][i]:
                                                            print(params['peA'][i])
                                                            print(params['peB'][i])
                                                            time_i = all_dens[i]['tauB']
                                                            time_k = all_pres[k]['Timestep']
                                                            time_q = all_pres_new2[q]['tauB']
                                                            time_s = all_pres_new3[s]['tauB']
                                                            time_t = all_pres_new[t]['tauB']

                                                            #Instantiate arrays for reading in interface pressure .txt file
                                                            final_time_i =np.array([])
                                                            final_align_press = np.array([])
                                                            align_press_total = 0
                                                            align_press_vals=0

                                                            #Instantiate arrays for reading in bulk pressure .txt file
                                                            bulk_time =np.array([])
                                                            bulk_press_time = np.array([])
                                                            bulk_press_total = 0
                                                            bulk_press_total_expand = 0
                                                            bulk_press_vals=0
                                                            shear_press_expand=0

                                                            #Instantiate arrays for reading in number density .txt file
                                                            bulk_nA_arr = np.array([])
                                                            bulk_nB_arr = np.array([])
                                                            bulk_n_arr = np.array([])
                                                            int_n_arr = np.array([])
                                                            int_nA_arr = np.array([])
                                                            int_nB_arr = np.array([])
                                                            gas_nA_arr = np.array([])
                                                            gas_nB_arr = np.array([])
                                                            dense_nA_arr = np.array([])
                                                            dense_nB_arr = np.array([])
                                                            dense_n_arr = np.array([])

                                                            #Instantiate arrays for reading in radial measurement .txt file
                                                            num_dens_arr_temp = np.zeros(50)
                                                            align_arr_temp = np.zeros(50)
                                                            press_arr_temp = np.zeros(50)
                                                            xA_arr_temp = np.zeros(50)
                                                            xB_arr_temp = np.zeros(50)
                                                            temp_count = np.zeros(50)
                                                            temp_count_count = np.zeros(50)

                                                            radius_arr_final = np.linspace(0,1.6,num=50)
                                                            num_dens_arr_final = np.zeros(50)
                                                            align_arr_final = np.zeros(50)
                                                            xA_arr_final = np.zeros(50)
                                                            xB_arr_final = np.zeros(50)
                                                            press_arr_final = np.zeros(50)
                                                            final_count = np.zeros(50)
                                                            final_count_count = np.zeros(50)

                                                            press_theory_time = np.array([])
                                                            press_theory_val = np.array([])
                                                            time_arr = np.array([])


                                                            #Loop over time in interface pressure .txt file
                                                            for j in range(0, len(time_i)):

                                                                #Current time in loop
                                                                time = time_i[j]

                                                                #Read values and set variables from interface pressure .txt file
                                                                align_press = all_dens[i]['press_align'][j]

                                                                if align_press > 0:
                                                                    final_time_i = np.append(final_time_i, all_dens[i]['tauB'].iloc[j])
                                                                    final_align_press = np.append(final_align_press, align_press)

                                                                    align_press_total += align_press
                                                                    align_press_vals += 1

                                                                #Read values and set variables from bulk pressure.txt files

                                                                time_id_k = np.where(time_k==time)[0][0]

                                                                bulk_trace = (all_pres[k]['bulkSigXX'].iloc[time_id_k]+all_pres[k]['bulkSigYY'].iloc[time_id_k])/2


                                                                bulk_press = bulk_trace / (all_pres[k]['bulkArea'].iloc[time_id_k])

                                                                bulk_trace_expand = (all_pres[k]['bulkSigXX'].iloc[time_id_k]+all_pres[k]['bulkSigXY'].iloc[time_id_k]+all_pres[k]['bulkSigYX'].iloc[time_id_k]+all_pres[k]['bulkSigYY'].iloc[time_id_k])/2
                                                                bulk_press_expand = bulk_trace_expand / (all_pres[k]['bulkArea'].iloc[time_id_k])

                                                                shear_stress = (all_pres[k]['bulkSigXY'].iloc[time_id_k]+all_pres[k]['bulkSigYX'].iloc[time_id_k])/2
                                                                shear_press = shear_stress / (all_pres[k]['bulkArea'].iloc[time_id_k])
                                                                if bulk_press>0:
                                                                    bulk_time = np.append(bulk_time, all_pres[k]['Timestep'].iloc[time_id_k])
                                                                    bulk_press_time = np.append(bulk_press_time, bulk_press/2)
                                                                    shear_press_expand +=shear_press
                                                                    bulk_press_total += bulk_press
                                                                    bulk_press_total_expand += bulk_press_expand
                                                                    bulk_press_vals += 1

                                                                #Read values and set variables from number density .txt files
                                                                time_id_q = np.where(time_q==time)[0]

                                                                bin_size = all_pres_new2[q]['sizeBin'][time_id_q]
                                                                bin_area = bin_size**2
                                                                bulk_nA_arr = np.append(bulk_nA_arr, all_pres_new2[q]['Na_bulk'][time_id_q] / (bin_area * all_pres_new2[q]['NBin_bulk'][time_id_q]))
                                                                bulk_nB_arr = np.append(bulk_nB_arr, all_pres_new2[q]['Nb_bulk'][time_id_q] / (bin_area * all_pres_new2[q]['NBin_bulk'][time_id_q]))
                                                                bulk_n_arr = np.append(bulk_n_arr, (all_pres_new2[q]['Nb_bulk'][time_id_q]+all_pres_new2[q]['Na_bulk'][time_id_q]) / (bin_area * all_pres_new2[q]['NBin_bulk'][time_id_q]))
                                                                int_n_arr = np.append(int_n_arr, (all_pres_new2[q]['Na_int'][time_id_q]+all_pres_new2[q]['Nb_int'][time_id_q]) / (bin_area * all_pres_new2[q]['NBin_int'][time_id_q]))
                                                                int_nA_arr = np.append(int_nA_arr, all_pres_new2[q]['Na_int'][time_id_q] / (bin_area * all_pres_new2[q]['NBin_int'][time_id_q]))
                                                                int_nB_arr = np.append(int_nB_arr, all_pres_new2[q]['Nb_int'][time_id_q] / (bin_area * all_pres_new2[q]['NBin_int'][time_id_q]))
                                                                gas_nA_arr = np.append(gas_nA_arr, all_pres_new2[q]['Na_gas'][time_id_q] / (bin_area * all_pres_new2[q]['NBin_gas'][time_id_q]))
                                                                gas_nB_arr = np.append(gas_nB_arr, all_pres_new2[q]['Nb_gas'][time_id_q] / (bin_area * all_pres_new2[q]['NBin_gas'][time_id_q]))
                                                                dense_n_arr = np.append(dense_n_arr, (all_pres_new2[q]['Na_int'][time_id_q]+all_pres_new2[q]['Nb_int'][time_id_q]+all_pres_new2[q]['Na_bulk'][time_id_q]+all_pres_new2[q]['Nb_bulk'][time_id_q]) / (bin_area * (all_pres_new2[q]['NBin_int'][time_id_q]+all_pres_new2[q]['NBin_bulk'][time_id_q])))
                                                                dense_nA_arr = np.append(dense_nA_arr, (all_pres_new2[q]['Na_int'][time_id_q]+all_pres_new2[q]['Na_bulk'][time_id_q]) / (bin_area * (all_pres_new2[q]['NBin_int'][time_id_q]+all_pres_new2[q]['NBin_bulk'][time_id_q])))
                                                                dense_nB_arr = np.append(dense_nB_arr, (all_pres_new2[q]['Nb_int'][time_id_q]+all_pres_new2[q]['Nb_bulk'][time_id_q]) / (bin_area * (all_pres_new2[q]['NBin_int'][time_id_q]+all_pres_new2[q]['NBin_bulk'][time_id_q])))

                                                                #Read values and set variables from radial measurement .txt files
                                                                time_id_s = np.where(time_s==time)[0]
                                                                time_id_t = np.where(time_t==time)[0]
                                                                
                                                                for y in time_id_s:
                                                                    if (len(time_id_s)>0) & (len(time_id_t)>0):

                                                                        if len(time_id_t)==1:
                                                                            edge_width = all_pres_new[t]['radius'][time_id_t[0]]
                                                                            nB = all_pres_new[t]['Nb'][time_id_t[0]]
                                                                            nA = all_pres_new[t]['Na'][time_id_t[0]]
                                                                        elif len(time_id_t)>1:
                                                                            edge_width = np.max(all_pres_new[t]['radius'][time_id_t])
                                                                            nB = np.max(all_pres_new[t]['Nb'][time_id_t])
                                                                            nA = np.max(all_pres_new[t]['Na'][time_id_t])
                                                                        if time not in time_arr:
                                                                            time_arr = np.append(time_arr, time)  
                                                                            if len(time_arr)>=2:

                                                                                for x in range(0, len(radius_arr_final)-1):
                                                                                    rad_where = np.where((radius_arr_final[x]<=radius_arr) & (radius_arr<radius_arr_final[x+1]))[0]
                                                                                    if len(rad_where)>0:
                                                                                        nonzero_align = np.where(align_arr[rad_where]!=0)[0]
                                                                                        if len(nonzero_align)>0:
                                                                                            press_arr_final[x]+=np.mean(press_arr[rad_where][nonzero_align])
                                                                                            align_arr_final[x]+=np.mean(align_arr[rad_where][nonzero_align])
                                                                                            num_dens_arr_final[x]+=np.mean(num_dens_arr[rad_where][nonzero_align])
                                                                                            xA_arr_final[x]+=np.mean(xA_arr[rad_where][nonzero_align])
                                                                                            xB_arr_final[x]+=np.mean(xB_arr[rad_where][nonzero_align])
                                                                                            final_count[x]+=1
                                                                                            final_count_count[x]+=len(rad_where)

                                                                                            press_arr_temp[x]=np.mean(press_arr[rad_where][nonzero_align])
                                                                                            align_arr_temp[x]=np.mean(align_arr[rad_where][nonzero_align])
                                                                                            xA_arr_temp[x]=np.mean(xA_arr[rad_where][nonzero_align])
                                                                                            xB_arr_temp[x]=np.mean(xB_arr[rad_where][nonzero_align])
                                                                                            num_dens_arr_temp[x]=np.mean(num_dens_arr[rad_where][nonzero_align])

                                                                                peNet_int = (int_nA_arr[-1]/int_n_arr[-1]) * float(params['peA'][i]) + (int_nB_arr[-1]/int_n_arr[-1]) * float(params['peB'][i])
                                                                                peNet = 0.5 * float(params['peA'][i]) + 0.5 * float(params['peB'][i])

                                                                                dens = densProbability(radius_arr_final, peNet, params['peA'][i])
                                                                                align = alignProbability(radius_arr_final, peNet, params['peA'][i])

                                                                                press = dens * align * ((xA_arr_temp * params['peA'][i] + xB_arr_temp * params['peB'][i]))
                                                                                press_int = 0
                                                                                for m in range(1, len(press)):
                                                                                    press_int += ((press[m-1]+press[m])/2)*(radius_arr_final[m]-radius_arr_final[m-1]) * edge_width
                                                                                press_theory_val = np.append(press_theory_val, press_int)
                                                                                press_theory_time = np.append(press_theory_time, time)


                                                                            radius_arr = np.array([])
                                                                            align_arr = np.array([])
                                                                            num_dens_arr = np.array([])
                                                                            press_arr = np.array([])
                                                                            xA_arr = np.array([])
                                                                            xB_arr = np.array([])
                                                                            radius_arr = np.append(radius_arr, all_pres_new3[s]['r_max'][y]/edge_width)   
                                                                            align_arr = np.append(align_arr, all_pres_new3[s]['align'][y])   
                                                                            xA_arr = np.append(xA_arr, all_pres_new3[s]['num_densA'][y] / all_pres_new3[s]['num_dens'][y])  
                                                                            xB_arr = np.append(xB_arr, all_pres_new3[s]['num_densB'][y] / all_pres_new3[s]['num_dens'][y])  
                                                                            num_dens_arr = np.append(num_dens_arr, all_pres_new3[s]['num_dens'][y])   
                                                                            press_arr = np.append(press_arr, all_pres_new3[s]['press'][y])   
                                                                        else:
                                                                            radius_arr = np.append(radius_arr, all_pres_new3[s]['r_max'][y]/edge_width)   
                                                                            area_slice = np.pi*(all_pres_new3[s]['r_max'][j]**2 - all_pres_new3[s]['r_min'][y]**2)
                                                                            align_arr = np.append(align_arr, all_pres_new3[s]['align'][y])   
                                                                            num_dens_arr = np.append(num_dens_arr, all_pres_new3[s]['num_dens'][y])   
                                                                            xA_arr = np.append(xA_arr, all_pres_new3[s]['num_densA'][y] / all_pres_new3[s]['num_dens'][y])  
                                                                            xB_arr = np.append(xB_arr, all_pres_new3[s]['num_densB'][y] / all_pres_new3[s]['num_dens'][y])  
                                                                            press_arr = np.append(press_arr, all_pres_new3[s]['press'][y]) 



                                                            #Time-average values for sufficiently long interface pressure .txt files
                                                            if align_press_vals > 50:        
                                                                avg_press = (align_press_total/align_press_vals)
                                                                align_press_arr = np.append(align_press_arr, avg_press)
                                                                align_peA=np.append(align_peA, params['peA'][i])
                                                                align_peB=np.append(align_peB, params['peB'][i])
                                                                align_peNet=np.append(align_peNet, params['peB'][i] * (1-params['xA'][i]/100) + params['peA'][i] * (params['xA'][i]/100))
                                                                align_xA=np.append(align_xA, params['xA'][i])
                                                                align_phi=np.append(align_phi, params['phi'][i])
                                                                align_eps=np.append(align_eps, params['eps'][i])

                                                            if bulk_press_vals > 50:        
                                                                avg_shear = np.append(avg_shear, shear_press_expand/bulk_press_vals)
                                                                avg_press = (bulk_press_total/(2*bulk_press_vals))
                                                                avg_press_expand = (bulk_press_total_expand/bulk_press_vals)
                                                                interpart_press=np.append(interpart_press, avg_press)
                                                                interpart_press_expand=np.append(interpart_press_expand, avg_press_expand)
                                                                interpart_peA=np.append(interpart_peA, params2['peA'][k])
                                                                interpart_peB=np.append(interpart_peB, params2['peB'][k])
                                                                interpart_peNet=np.append(interpart_peNet, params2['peB'][k] * (1-params2['xA'][k]/100) + params2['peA'][k] * (params2['xA'][k]/100))
                                                                interpart_xA=np.append(interpart_xA, params2['xA'][k])
                                                                interpart_phi=np.append(interpart_phi, params2['phi'][k])
                                                                interpart_eps=np.append(interpart_eps, params2['eps'][k])

                                                            for m in range(0, len(align_arr_temp)):
                                                                if temp_count[m]>0:
                                                                    align_arr_final[m]=align_arr_final[m]/final_count[m]
                                                                    num_dens_arr_final[m]=num_dens_arr_final[m]/final_count[m]
                                                                    xA_arr_final[m]=xA_arr_final[m]/final_count[m]
                                                                    xB_arr_final[m]=xB_arr_final[m]/final_count[m]
                                                            
                                                            peNet_int = (int_nA_arr/int_n_arr) * float(params['peA'][i]) + (int_nB_arr/int_n_arr) * float(params['peB'][i])

                                                            latNet_int = np.zeros(len(peNet_int))
                                                            for z in range(0, len(peNet_int)):
                                                                latNet_int[z] = conForRClust(peNet_int[z]-50, 1.0)

                                                            theoryss = (peNet_int-50) * 2 * 1.874 * np.sqrt(3) / latNet_int
                
                                                            fsize=10
                                                            mpl.rcParams.update({'font.size': 13})
                                                            mkSz = [0.1, 0.1, 0.15, 0.1, 0.1]
                                                            msz=40
                                                            yellow = ("#fec44f")
                                                            green = ("#77dd77")
                                                            red = ("#ff6961")


                                                            nonzero_bulk = np.where(bulk_press_time>0)[0]
                                                            nonzero_align = np.where(final_align_press>0)[0]

                                                            if len(nonzero_bulk)>10:
                                                                if len(nonzero_align)>10:
                                                                    
                                                                    fastSim = '#de2d26'
                                                                    fastTheory = '#fc9272'
                                                                    slowSim = '#3182bd'
                                                                    slowTheory = '#9ecae1'
                                                                    fig, ax1 = plt.subplots(figsize=(12,5))
                                                                    interpart_max = np.max(bulk_press_time[nonzero_bulk][10:])
                                                                    align_max = np.max(final_align_press[nonzero_align][10:])
                                                                    if align_max >= interpart_max:
                                                                        plot_max = align_max + 500
                                                                    else:
                                                                        plot_max = interpart_max + 500

                                                                    interpart_min = np.min(bulk_press_time[nonzero_bulk][10:])
                                                                    align_min = np.min(final_align_press[nonzero_align][10:])
                                                                    if align_min <= interpart_min:
                                                                        plot_min = align_min - 500
                                                                    else:
                                                                        plot_min = interpart_min - 500


                                                                    plt.plot(bulk_time[nonzero_bulk], bulk_press_time[nonzero_bulk],
                                                                                   c=slowSim, lw=1.8, ls='--', label='Sim Bulk')
                                                                    plt.plot(time_i[nonzero_align], final_align_press[nonzero_align],
                                                                                   c=fastSim, lw=1.8, ls='--', label='Sim Interface')
                                                                    print(len(time_q))
                                                                    print(len(time_i))
                                                                    print(len(theoryss))
                                                                    if len(time_q)==len(theoryss):
                                                                        plt.plot(time_q, theoryss,
                                                                                   c=slowTheory, lw=1.8, ls='dotted', label='Theory Bulk')
                                                                    elif len(time_i)==len(theoryss):
                                                                        plt.plot(time_i, theoryss,
                                                                                   c=slowTheory, lw=1.8, ls='dotted', label='Theory Bulk')
                                                                    plt.plot(press_theory_time, press_theory_val,
                                                                                   c=fastTheory, lw=1.8, ls='dotted', label='Theory Interface')
                                                                    ax1.set_xlim(0, 100)        
                                                                    # y limits


                                                                    ax1.set_ylim(plot_min, plot_max) 


                                                                    ax1.set_xlabel(r'time ($\tau$)', fontsize=fsize*2.8)



                                                                    ax1.set_ylabel(r'Pressure ($\Pi$)', fontsize=fsize*2.8)


                                                                    # Set all the x ticks for radial plots
                                                                    loc = ticker.MultipleLocator(base=20)
                                                                    ax1.xaxis.set_major_locator(loc)
                                                                    loc = ticker.MultipleLocator(base=10)
                                                                    ax1.xaxis.set_minor_locator(loc)


                                                                    # Set y ticks
                                                                    loc = ticker.MultipleLocator(base=400)
                                                                    ax1.yaxis.set_major_locator(loc)
                                                                    loc = ticker.MultipleLocator(base=200)
                                                                    ax1.yaxis.set_minor_locator(loc)
                                                                    # Left middle plot
                                                                    plt.legend(loc='upper right', fontsize=fsize*2.0)

                                                                    ax1.tick_params(axis='x', labelsize=fsize*2.5)
                                                                    ax1.tick_params(axis='y', labelsize=fsize*2.5)

                                                                    plt.tight_layout()
                                                                    plt.savefig('/Volumes/External/pressure_time2/random_pa' + str(int(params['peA'][i])) + '_pb'+str(int(params['peB'][i])), dpi=300)
                                                                    plt.close()
                                                                    
                                                            
                                                            
                                                                
                                                                
                                                                
                                                                
                                                                
                                                                
                                        
                                                
        
        
    
'''
                                                                    
                                                            print(len(xA_arr_final))
                                                            print(len(press_arr))
                                                            print(len(time_id2))
                                                            plt.plot(press_theory_time, press_theory_val)
                                                            plt.show()
                                                            plt.plot(radius_arr_final, xA_arr_temp, color='blue')
                                                            plt.plot(radius_arr_final, xB_arr_temp, color='red')
                                                            plt.plot(radius_arr_final, num_dens_arr_temp, color='green')
                                                            plt.show()
                                                            stop
                                                            print(params['peA'][i])
                                                            print(params['peB'][i])
                                                            print(paramss['peA'][m])
                                                            print(paramss['peB'][m])

                                                            peNet_int = (((int_nA_arr/int_n_arr) * float(params['peA'][i]) + (int_nB_arr/int_n_arr) * float(params['peB'][i])))
                                                            peNet_ints = (((int_nA_arrs/int_n_arrs) * float(paramss['peA'][m]) + (int_nB_arrs/int_n_arrs) * float(paramss['peB'][m])))

                                                            peNet = params2['peB'][k] * (1-params2['xA'][k]/100) + params2['peA'][k] * (params2['xA'][k]/100)
                                                            latNet_int = np.zeros(len(peNet_int))
                                                            for z in range(0, len(peNet_int)):
                                                                latNet_int[z] = getLat(peNet_int[z] - 50, 1.0)
                                                            latNet_ints = np.zeros(len(peNet_ints))
                                                            for z in range(0, len(peNet_ints)):
                                                                latNet_ints[z] = getLat(peNet_ints[z] - 50, 1.0)

                                                            theoryss = (peNet_int-50) * 2 * 1.92 * np.sqrt(3) / latNet_int
                                                            theorysss = (peNet_ints-50) * 2 * 1.92 * np.sqrt(3) / latNet_ints


                                                            x_range2 = np.array([10000000, 20000000])
                                                            y_range2 = np.array([theoryss, theoryss])

                                                            x_range_int = np.array([0.3, 0.3+0.0000000000001])
                                                            y_range_int = np.array([-1000000, 1000000])

                                                            fsize=10
                                                            mpl.rcParams.update({'font.size': 13})
                                                            mkSz = [0.1, 0.1, 0.15, 0.1, 0.1]
                                                            msz=40
                                                            yellow = ("#fec44f")
                                                            green = ("#77dd77")
                                                            red = ("#ff6961")


                                                            nonzero_bulk = np.where(bulk_press_time>0)[0]
                                                            nonzero_align = np.where(align_press_time>0)[0]
                                                            nonzero_bulks = np.where(bulk_press_times>0)[0]
                                                            nonzero_aligns = np.where(align_press_times>0)[0]
                                                            if len(nonzero_bulk)>10:
                                                                if len(nonzero_align)>10:
                                                                    fig, ax1 = plt.subplots(figsize=(12,5))
                                                                    interpart_max = np.max(bulk_press_time[nonzero_bulk][10:])
                                                                    align_max = np.max(align_press_time[nonzero_align][10:])
                                                                    if align_max >= interpart_max:
                                                                        plot_max = align_max + 2000
                                                                    else:
                                                                        plot_max = interpart_max + 2000

                                                                    interpart_min = np.min(bulk_press_time[nonzero_bulk][10:])
                                                                    align_min = np.min(align_press_time[nonzero_align][10:])
                                                                    if align_min <= interpart_min:
                                                                        plot_min = align_min - 2000
                                                                    else:
                                                                        plot_min = interpart_min - 2000


                                                                    plt.plot(bulk_time[nonzero_bulk], bulk_press_time[nonzero_bulk],
                                                                                   c=slowCol, lw=1.8*1.8, ls='--')
                                                                    plt.plot(align_time[nonzero_align], align_press_time[nonzero_align],
                                                                                   c=fastCol, lw=1.8*1.8, ls='--')
                                                                    plt.plot(time_arr3, theoryss,
                                                                                   c=green, lw=1.8*1.8, ls='-', label='Theory')
                                                                    ax1.set_xlim(0, 200)        
                                                                    # y limits


                                                                    ax1.set_ylim(plot_min, plot_max) 


                                                                    ax1.set_xlabel(r'time ($\tau$)', fontsize=fsize*2.8)



                                                                    ax1.set_ylabel(r'Pressure ($\Pi$)', fontsize=fsize*2.8)


                                                                    # Set all the x ticks for radial plots
                                                                    loc = ticker.MultipleLocator(base=20)
                                                                    ax1.xaxis.set_major_locator(loc)
                                                                    loc = ticker.MultipleLocator(base=10)
                                                                    ax1.xaxis.set_minor_locator(loc)


                                                                    # Set y ticks
                                                                    loc = ticker.MultipleLocator(base=1000)
                                                                    ax1.yaxis.set_major_locator(loc)
                                                                    loc = ticker.MultipleLocator(base=500)
                                                                    ax1.yaxis.set_minor_locator(loc)
                                                                    # Left middle plot
                                                                    plt.legend(loc='upper right', fontsize=fsize*2.7)

                                                                    ax1.tick_params(axis='x', labelsize=fsize*2.5)
                                                                    ax1.tick_params(axis='y', labelsize=fsize*2.5)

                                                                    plt.tight_layout()
                                                                    plt.show()






                                                            nonzero_bulk = np.where(bulk_press_time>0)[0]
                                                            nonzero_align = np.where(align_press_time>0)[0]
                                                            nonzero_bulks = np.where(bulk_press_times>0)[0]
                                                            nonzero_aligns = np.where(align_press_times>0)[0]
                                                            if len(nonzero_bulks)>10:
                                                                if len(nonzero_aligns)>10:
                                                                    fig, ax1 = plt.subplots(figsize=(12,5))
                                                                    interpart_max = np.max(bulk_press_times[nonzero_bulks][10:])
                                                                    align_max = np.max(align_press_times[nonzero_aligns][10:])
                                                                    if align_max >= interpart_max:
                                                                        plot_max = align_max + 2000
                                                                    else:
                                                                        plot_max = interpart_max + 2000

                                                                    interpart_min = np.min(bulk_press_times[nonzero_bulks][10:])
                                                                    align_min = np.min(align_press_times[nonzero_aligns][10:])
                                                                    if align_min <= interpart_min:
                                                                        plot_min = align_min - 2000
                                                                    else:
                                                                        plot_min = interpart_min - 2000

                                                                    plt.plot(bulk_times[nonzero_bulks], bulk_press_times[nonzero_bulks],
                                                                                   c=fastCol, lw=1.8*1.8, ls='-', label='Bulk')
                                                                    plt.plot(align_times[nonzero_aligns], align_press_times[nonzero_aligns],
                                                                                   c=slowCol, lw=1.8*1.8, ls='-', label='Interface')
                                                                    plt.plot(time_arr3s, theorysss,
                                                                                   c=green, lw=1.8*1.8, ls='-', label='Theory')

                                                                    ax1.set_xlim(0, 200)        
                                                                    # y limits


                                                                    ax1.set_ylim(plot_min, plot_max) 


                                                                    ax1.set_xlabel(r'time ($\tau$)', fontsize=fsize*2.8)



                                                                    ax1.set_ylabel(r'Pressure ($\Pi$)', fontsize=fsize*2.8)


                                                                    # Set all the x ticks for radial plots
                                                                    loc = ticker.MultipleLocator(base=20)
                                                                    ax1.xaxis.set_major_locator(loc)
                                                                    loc = ticker.MultipleLocator(base=10)
                                                                    ax1.xaxis.set_minor_locator(loc)


                                                                    # Set y ticks
                                                                    loc = ticker.MultipleLocator(base=1000)
                                                                    ax1.yaxis.set_major_locator(loc)
                                                                    loc = ticker.MultipleLocator(base=500)
                                                                    ax1.yaxis.set_minor_locator(loc)
                                                                    # Left middle plot
                                                                    plt.legend(loc='upper right', fontsize=fsize*2.7)

                                                                    ax1.tick_params(axis='x', labelsize=fsize*2.5)
                                                                    ax1.tick_params(axis='y', labelsize=fsize*2.5)

                                                                    plt.tight_layout()
                                                                    plt.show()
                                                                        

    
'''
import matplotlib.patches as mpatches
from matplotlib import cm
import matplotlib.patches as patches

stop
        
fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(111)
div_min = -3
min_n = 0
max_n = 20
levels_text=40
level_boundaries = np.linspace(min_n, max_n, levels_text + 1)
tick_locs   = [0.0,np.pi/6,np.pi/3]
tick_labels = ['0',r'$\pi/6$',r'$\pi/3$']

im = plt.scatter(first_interpart_peNet, first_bulk_press, linewidths=1.0, edgecolor='black', facecolor='green', s=60.0)
im = plt.scatter(first_align_peNet, first_align_press, linewidths=1.0, edgecolor='black', facecolor='yellow', s=60.0)
         
yellow_patch = mpatches.Patch(color='yellow', label='Fast Interface')
red_patch = mpatches.Patch(color='green', label='Slow Bulk')
plt.legend(handles=[yellow_patch, red_patch], fancybox=True, framealpha=0.75, ncol=1, fontsize=12, loc='upper left',labelspacing=0.1, handletextpad=0.1)

    
plt.xlabel(r'$\mathrm{Pe}_\mathrm{Net}$', fontsize=20)
plt.ylabel(r'Initial Pressure ($\Pi$)', fontsize=20)
plt.tight_layout()
plt.show()

fig = plt.figure(figsize=(10,7))
ax = fig.add_subplot(111)
div_min = -3
min_n = 0
max_n = 20
levels_text=40
level_boundaries = np.linspace(min_n, max_n, levels_text + 1)
tick_locs   = [0.0,np.pi/6,np.pi/3]
tick_labels = ['0',r'$\pi/6$',r'$\pi/3$']
for i in range(0, len(first_bulk_press)):
    for j in range(0, len(press_fast_dense)):
        if (first_interpart_peA[i]==pa_pair[j]) and (first_interpart_peB[i]==pb_pair[j]):
            im = plt.scatter(first_interpart_peNet[i], ((first_bulk_press[i] - press_slow_dense[j])/press_slow_dense[j])*100, linewidths=1.0, edgecolor='black', facecolor='green', s=60.0)
        elif (first_interpart_peB[i]==pa_pair[j]) and (first_interpart_peA[i]==pb_pair[j]):
            im = plt.scatter(first_interpart_peNet[i], ((first_bulk_press[i] - press_slow_dense[j])/press_slow_dense[j])*100, linewidths=1.0, edgecolor='black', facecolor='green', s=60.0)
for i in range(0, len(first_align_press)):
    for j in range(0, len(press_slow_int)):
        if (first_align_peA[i]==pa_pair[j]) and (first_align_peB[i]==pb_pair[j]):
            im = plt.scatter(first_align_peNet[i], ((first_align_press[i]-press_fast_int[j])/press_fast_int[j])*100, linewidths=1.0, edgecolor='black', facecolor='yellow', s=60.0)
        elif (first_align_peB[i]==pa_pair[j]) and (first_align_peA[i]==pb_pair[j]):
            im = plt.scatter(first_align_peNet[i], ((first_align_press[i]-press_fast_int[j])/press_fast_int[j])*100, linewidths=1.0, edgecolor='black', facecolor='yellow', s=60.0)
yellow_patch = mpatches.Patch(color='yellow', label='Fast Interface')
red_patch = mpatches.Patch(color='green', label='Slow Bulk')
plt.legend(handles=[yellow_patch, red_patch], fancybox=True, framealpha=0.75, ncol=1, fontsize=12, loc='upper left',labelspacing=0.1, handletextpad=0.1)

    
plt.xlabel(r'$\mathrm{Pe}_\mathrm{Net}$', fontsize=20)
plt.ylabel('Percent Error (%)', fontsize=20)
plt.tight_layout()
plt.show()

plt.figure(figsize=(8,6))
plt.scatter(first_interpart_peNet, first_bulk_press, s=10.0, color='blue', label='bulk interparticle')
plt.scatter(first_align_peNet, first_align_press, s=10.0, color='red', label='aligned interface')


plt.title('fast interface, slow bulk (stable)', fontsize=22)
plt.ylabel(r'Initial Pressure ($\Pi$)')
plt.xlabel(r'$\mathrm{Pe}_\mathrm{Net}$')
plt.legend(loc='upper left')
plt.ylim([-2500, 25000])
plt.show() 
stop

plt.figure(figsize=(8,6))
plt.scatter(interpart_peNet, interpart_press, c='blue', label='bulk interparticle')
plt.scatter(align_peNet, align_press_arr, c='red', label='aligned interface')
plt.ylabel(r'Pressure ($\Pi$)')
plt.xlabel(r'$\mathrm{Pe}_\mathrm{Net}$')
plt.legend()
plt.show()

    
'''
for i in range(0, len(align_peA)):  
    for j in range(0, len(align_peB)):  
        for k in range(0, len(interpart_peA)):  
            for l in range(0, len(interpart_peB)):  
                if (align_peA[i]==interpart_peA[k]) & (align_peB[j]==interpart_peB[l]):
                    
'''

100
100
372
999
372
100
150
442
999
442
100
200
999
999
999
100
250
999
999
999
100
350
999
999
999
100
450
999
27
27
150
150
188
999
188
150
200
999
999
999
150
250
999
999
999
150
350
999
999
999
150
450
999
14
14
200
350
999
999
999
200
450
999
19
19
200
200
999
999
999
200
250
999
999
999
250
450
999
23
23
250
250
999
999
999
250
350
999
999
999
350
350
999
999
999
350
450
999
17
17
450
450
999
15
15


NameError: name 'stop' is not defined